In [1]:
import os,sys
import numpy as np
import pandas as pd

from keras.preprocessing import image
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.layers import Dropout
# from PIL import ImageFile
from keras.applications.resnet50 import ResNet50
import shutil
from keras.models import model_from_json

Using TensorFlow backend.


In [28]:
Images=pd.read_csv("DetectionStats.csv")
Classes=pd.read_csv('DataStats.csv')
ImagePath=Images['path to crop']
Sizes=Images[['width','height']]
Class=Images['class']

In [14]:
def build_ResNet50(input_tensor_shape,output_shape):
    '''
    # reference 
        https://keras.io/applications/#vgg16
        https://www.tensorflow.org/api_docs/python/tf/contrib/keras/applications/ResNet50
    # model defination
        https://github.com/tensorflow/tensorflow/blob/r1.2/tensorflow/contrib/keras/python/keras/applications/resnet50.py
        
    # Arguments
        include_top: whether to include the fully-connected layer at the top of the network.
     
    '''
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape= input_tensor_shape)
    
    x_model = base_model.output
    
    x_model = GlobalAveragePooling2D(name='globalaveragepooling2d')(x_model)
    
    x_model = Dense(1024, activation='relu',name='fc1_Dense')(x_model)
    x_model = Dropout(0.5, name='dropout_1')(x_model)
    
    x_model = Dense(256, activation='relu',name='fc2_Dense')(x_model)
    x_model = Dropout(0.5, name='dropout_2')(x_model)
    predictions = Dense(output_shape, activation='sigmoid',name='output_layer')(x_model)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    return model


In [15]:
def SaveModel(model,Path):
    # serialize model to JSON
    model_json = model.to_json()
    if os.path.isdir(Path):
        shutil.rmtree(Path)
    os.mkdir(Path)
    with open(Path+"/model_architecture.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(Path+"/model_weights.h5")
    print("Saved model to ",Path)

def loadModel(Path):
    # load json and create model
    json_file = open(Path+'/model_architecture.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(Path+"/model_weights.h5")
    print("Loaded model from disk",Path)
    return loaded_model

In [16]:
# get model
img_rows, img_cols, img_channel = 224, 224, 3
input_tensor_shape=(img_rows, img_cols, img_channel)
output_shape=Classes.shape[0]
model=loadModel('Resnet50/pretrained')

('Loaded model from disk', 'Resnet50/Pretrained')


In [17]:
print("No of layers:",len(model.layers))
print(model.summary())

('No of layers:', 181)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
______________________________________________________________________________________

In [18]:
#FreezeLayers
for layer in model.layers[:175]:
        layer.trainable = False
for layer in model.layers[175:]:
    layer.trainable = True

In [109]:
import cv2
Images = [cv2.imread(Path) for Path in ImagePath]
x_test=[cv2.resize(image,(224,224)) for i,image in enumerate(Images) if i%10==0]
x_valid=[cv2.resize(image,(224,224)) for i,image in enumerate(Images) if i%10==1 or i%10==2]
x_train=[cv2.resize(image,(224,224)) for i,image in enumerate(Images) if i%10>2]
x_train=np.stack(x_train,axis=0)
x_valid=np.stack(x_valid,axis=0)
x_test=np.stack(x_test,axis=0)
print(x_train.shape,x_valid.shape,x_test.shape)
y_test1=[j for i,j in enumerate(Class) if i%10==0]
y_valid1=[j for i,j in enumerate(Class) if i%10==1 or i%10==2]
y_train1=[j for i,j in enumerate(Class) if i%10>2]
print(len(y_train1),len(y_valid1),len(y_test1))

((45, 224, 224, 3), (14, 224, 224, 3), (7, 224, 224, 3))
(45, 14, 7)


In [92]:
# compile the model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

# set train Generator
datagen = ImageDataGenerator(rotation_range=30,width_shift_range=0.2,height_shift_range=0.2,horizontal_flip=True)
datagen.fit(x_train)


In [110]:
def onehot(encoded_Y):
    TrainY=np.zeros((encoded_Y.shape[0],encoder.classes_.shape[0]))
    for i in range(encoded_Y.shape[0]):
        TrainY[i,encoded_Y[i]]=1
    return TrainY

In [111]:
from sklearn import preprocessing
from keras.utils import to_categorical, np_utils
encoder = preprocessing.LabelEncoder()
encoder.fit(Classes['Bird Class'].values)
print(encoder.classes_)
y_train=onehot(encoder.transform(y_train1))
y_valid=onehot(encoder.transform(y_valid1))
y_test=onehot(encoder.transform(y_test1))

['Bulbul' 'Collared_dove' 'Common_sparrow' 'Coppersmith' 'Crow'
 'Crow_pheasant' 'Drongo' 'Golden_backed_woodpecker' 'Green Barbet'
 'Hoopoe' 'Indian_hornbill' 'Indian_robin' 'Jungle_babbler' 'Koel'
 'Little_green_beeeater' 'Magpie_robin' 'Myna' 'Owlet' 'Parakeet'
 'Pariah_kite' 'Partridge' 'Peacock' 'Pied_wagtail' 'Pigeon' 'Pond_heron'
 'Red_wattled_lapwing' 'Rufous_backed_shrike' 'Shikra' 'Sunbird'
 'Tailor_bird' 'Tree_pie' 'White_breasted_kingfisher'
 'White_breasted_water_hen']


In [113]:
# trainning process
nb_epoch = sys.argv[1]
batch_size = sys.argv[2]
checkpointer = ModelCheckpoint(filepath= './ResNet50_weights1.hdf5', verbose=1, monitor='val_acc',save_best_only=True, save_weights_only=True)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch = x_train.shape[0],
                    epochs=nb_epoch,
                    validation_data = (x_valid, y_valid),
                    callbacks=[checkpointer])

Epoch 1/5
45/45 [==============================] - 430s 10s/step - loss: 0.8776 - acc: 0.4858 - val_loss: 0.7512 - val_acc: 0.4719

Epoch 00001: val_acc improved from -inf to 0.47186, saving model to ./ResNet50_weights1.hdf5
Epoch 2/5
45/45 [==============================] - 432s 10s/step - loss: 0.7942 - acc: 0.5333 - val_loss: 0.6653 - val_acc: 0.5801

Epoch 00002: val_acc improved from 0.47186 to 0.58009, saving model to ./ResNet50_weights1.hdf5
Epoch 3/5
45/45 [==============================] - 425s 9s/step - loss: 0.7181 - acc: 0.5864 - val_loss: 0.5959 - val_acc: 0.6623

Epoch 00003: val_acc improved from 0.58009 to 0.66234, saving model to ./ResNet50_weights1.hdf5
Epoch 4/5
45/45 [==============================] - 434s 10s/step - loss: 0.6608 - acc: 0.6301 - val_loss: 0.5365 - val_acc: 0.7727

Epoch 00004: val_acc improved from 0.66234 to 0.77273, saving model to ./ResNet50_weights1.hdf5
Epoch 5/5
45/45 [==============================] - 416s 9s/step - loss: 0.6110 - acc: 0.6701

In [114]:
ls

AllData.csv                       Resnet50/
Annotations/                      Resnet50_preprocess.ipynb
BirdDetection.ipynb               Resnet50_pretrained.ipynb
BirdDetection.py                  Resnet50_train.ipynb
Cropped/                          Rufous_backed_shrike/
Dark-necked tailorbird/           Tailor bird/
Data/                             Untitled.ipynb
DataStats.csv                     Untitled1.ipynb
DetectionStats.csv                Untitled2.ipynb
Eurasian Sparrowhawk/             data.py
Jungle crow/                      project1.pgsql
MobileNetSSD_deploy.caffemodel    ssd_keras/
MobileNetSSD_deploy.prototxt.txt  untitled folder/
ResNet50_weights1.hdf5


In [117]:
model=loadModel('Resnet50/Pretrained')
model.load_weights('ResNet50_weights1.hdf5')

('Loaded model from disk', 'Resnet50/Pretrained')


In [121]:
Predictions=model.predict(x_test)

In [127]:
np.argmax(Predictions,axis=1)

array([ 1, 32, 24, 31, 31, 31, 31])

In [130]:
onehot(np.array([1,2,30]))

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0.]])

In [133]:
encoder.transform(y_train1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [134]:
Predictions

array([[0.63463277, 0.64164954, 0.2314926 , 0.30753452, 0.24241287,
        0.5842209 , 0.2133135 , 0.3243978 , 0.3420515 , 0.51302826,
        0.34139395, 0.36484686, 0.57096034, 0.37693858, 0.24864772,
        0.40823764, 0.40572783, 0.37619182, 0.15574186, 0.5405035 ,
        0.33881336, 0.33536327, 0.32032183, 0.11497447, 0.5834245 ,
        0.33970132, 0.49647155, 0.46924657, 0.33417457, 0.18183869,
        0.07023387, 0.5504544 , 0.57829577],
       [0.37492612, 0.5591103 , 0.15588406, 0.3053024 , 0.29795396,
        0.41748196, 0.08454087, 0.3137441 , 0.50168085, 0.2617175 ,
        0.25889006, 0.19374923, 0.5973143 , 0.45275274, 0.17985846,
        0.3895745 , 0.5157075 , 0.32223314, 0.2696718 , 0.6516947 ,
        0.37357557, 0.38632214, 0.24699144, 0.1361026 , 0.34089202,
        0.3455455 , 0.52953285, 0.21347794, 0.37329504, 0.16284284,
        0.09813202, 0.55985135, 0.7039533 ],
       [0.57862735, 0.36208922, 0.17961346, 0.40779397, 0.4186151 ,
        0.3873914 , 0.1328